In [8]:
!virtualenv bertenv

Using base prefix 'c:\\users\\benar\\anaconda3'
  No LICENSE.txt / LICENSE found in source
New python executable in C:\Users\benar\Desktop\project_data\bertenv\Scripts\python.exe
Installing setuptools, pip, wheel...
done.


In [17]:
# libraries
import os
import csv
import time
import tensorflow as tf
import bert
import pandas as pd
from NYT_parser import NYTArticle
from utilities import *

In [18]:
# get pathways
train_paths = data_setup.get_paths('train')
article = NYTArticle.from_file(os.path.join('data', train_paths[0][0]))

In [20]:
# set destination folder and make folders if needed
dest_folder = 'sample/train/'
if not os.path.exists(dest_folder):
    os.makedirs(dest_folder)

In [4]:
# attempt a single file output
print("Writing", dest_folder+article.docid)
article.simple_csv_output(dest_folder)

Writing transdata/train/739212


In [9]:
# attempt 50 article write minus filtered out
for path in train_paths[:50]:
    article = NYTArticle.from_file(os.path.join('data',path[0]))
    if article.pass_filters():
        print("Writing", dest_folder+article.docid)
        article.simple_csv_output(dest_folder)

Writing transdata/train/1397928
Writing transdata/train/1609775
Writing transdata/train/297100
Writing transdata/train/448007
Writing transdata/train/1463610
Writing transdata/train/590096
Writing transdata/train/762936
Writing transdata/train/1570877
Writing transdata/train/1725268
Writing transdata/train/1647864
Writing transdata/train/1251662
Writing transdata/train/551924
Writing transdata/train/74723
Writing transdata/train/1535917
Writing transdata/train/744457
Writing transdata/train/97129
Writing transdata/train/899133
Writing transdata/train/791668
Writing transdata/train/779448
Writing transdata/train/1287650
Writing transdata/train/927546
Writing transdata/train/1587613
Writing transdata/train/638654
Writing transdata/train/870914
Writing transdata/train/800527
Writing transdata/train/1584905
Writing transdata/train/1287373
Writing transdata/train/1337862
Writing transdata/train/1646102
Writing transdata/train/1363692
Writing transdata/train/50986
Writing transdata/train/148

In [11]:
# write all filtered articles in train set
opened_article = 0
written_article = 0
start = time.time()
for path in train_paths:
    opened_article += 1
    article = NYTArticle.from_file(os.path.join('data',path[0]))
    if article.pass_filters():
        written_article += 1
        article.simple_csv_output(dest_folder)
    if opened_article % 10000 == 0:
        print("Opened", opened_article,"articles. Output", written_article,"articles.")
print(f"\n... processed {opened_article} files in {time.time() - start} seconds")


... processed 1001 files in 2.489995241165161 seconds


In [21]:
# build dataframe with articles
sample = 1000

headlines = []
bodies = []

for path in train_paths[:sample]:
    article = NYTArticle.from_file(os.path.join('data',path[0]))
    if article.pass_filters():
        article.simple_csv_output(dest_folder)
        headlines.append(article.print_hede[0])
        # join paragraphs for now, may want to do something else later
        bodies.append('\n'.join(article.paragraphs))

train_df = pd.DataFrame(data={'headline':headlines, 'body':bodies})
train_df.head()

,headline,body
0,"A Killer, Yes, but She's a Good Old Girl",Aileen Wuornos was big news 10 years ago. At a...
1,"How High's The Bidding, Mama?",THE blue Rolls Royce sat at the York Street cu...
2,Pacificorp reports earnings for Qtr to Sept 30,LEAD:\n*3*** COMPANY REPORTS **\n*3* Pacificor...
3,Stewart Set to Return,There is bad news for Oakland's competition in...
4,Taxing Online Sales,"To the Editor:\nRe ''So Many Online Sales, So ..."


## Universal Transformer

In [22]:
# Install deps
!pip install -q -U tensor2tensor
!pip install -q tensorflow matplotlib

In [27]:
# Imports we need.
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import collections

from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics

# Enable TF Eager execution
tfe = tf.contrib.eager
tfe.enable_eager_execution()

# Other setup
Modes = tf.estimator.ModeKeys

# Setup some directories
data_dir = os.path.expanduser("./t2t/data")
tmp_dir = os.path.expanduser("./t2t/tmp")
train_dir = os.path.expanduser("./t2t/train")
checkpoint_dir = os.path.expanduser("./t2t/checkpoints")
tf.gfile.MakeDirs(data_dir)
tf.gfile.MakeDirs(tmp_dir)
tf.gfile.MakeDirs(train_dir)
tf.gfile.MakeDirs(checkpoint_dir)
gs_data_dir = "gs://tensor2tensor-data/"
gs_ckpt_dir = "gs://tensor2tensor-checkpoints/"

In [24]:
problems.available()

['algorithmic_addition_binary40',
 'algorithmic_addition_decimal40',
 'algorithmic_cipher_shift200',
 'algorithmic_cipher_shift5',
 'algorithmic_cipher_vigenere200',
 'algorithmic_cipher_vigenere5',
 'algorithmic_identity_binary40',
 'algorithmic_identity_decimal40',
 'algorithmic_math_deepmind_all',
 'algorithmic_math_two_variables',
 'algorithmic_multiplication_binary40',
 'algorithmic_multiplication_decimal40',
 'algorithmic_reverse_binary40',
 'algorithmic_reverse_binary40_test',
 'algorithmic_reverse_decimal40',
 'algorithmic_reverse_nlplike32k',
 'algorithmic_reverse_nlplike8k',
 'algorithmic_shift_decimal40',
 'algorithmic_sort_problem',
 'audio_timit_characters_tune',
 'audio_timit_tokens8k_test',
 'audio_timit_tokens8k_tune',
 'babi_qa_concat_all_tasks_10k',
 'babi_qa_concat_all_tasks_1k',
 'babi_qa_concat_task10_10k',
 'babi_qa_concat_task10_1k',
 'babi_qa_concat_task11_10k',
 'babi_qa_concat_task11_1k',
 'babi_qa_concat_task12_10k',
 'babi_qa_concat_task12_1k',
 'babi_qa_con

In [28]:
# Fetch the problem
ende_problem = problems.problem("translate_ende_wmt32k")

# Copy the vocab file locally so we can encode inputs and decode model outputs
# All vocabs are stored on GCS
vocab_name = "vocab.translate_ende_wmt32k.32768.subwords"
vocab_file = os.path.join(gs_data_dir, vocab_name)
!gsutil cp {vocab_file} {data_dir}

# Get the encoders from the problem
encoders = ende_problem.feature_encoders(data_dir)

# Setup helper functions for encoding and decoding
def encode(input_str, output_str=None):
  """Input str to features dict, ready for inference"""
  inputs = encoders["inputs"].encode(input_str) + [1]  # add EOS id
  batch_inputs = tf.reshape(inputs, [1, -1, 1])  # Make it 3D.
  return {"inputs": batch_inputs}

def decode(integers):
  """List of ints to str"""
  integers = list(np.squeeze(integers))
  if 1 in integers:
    integers = integers[:integers.index(1)]
  return encoders["inputs"].decode(np.squeeze(integers))

Copying gs://tensor2tensor-data/vocab.translate_ende_wmt32k.32768.subwords...
/ [0 files][    0.0 B/313.8 KiB]                                                
/ [1 files][313.8 KiB/313.8 KiB]                                                

Operation completed over 1 objects/313.8 KiB.                                    
W0713 14:17:29.701812  4136 deprecation_wrapper.py:119] From C:\Users\benar\Anaconda3\lib\site-packages\tensor2tensor\data_generators\text_encoder.py:940: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.



In [ ]:
# # Generate and view the data
# # This cell is commented out because WMT data generation can take hours

# ende_problem.generate_data(data_dir, tmp_dir)
# example = tfe.Iterator(ende_problem.dataset(Modes.TRAIN, data_dir)).next()
# inputs = [int(x) for x in example["inputs"].numpy()] # Cast to ints.
# targets = [int(x) for x in example["targets"].numpy()] # Cast to ints.



# # Example inputs as int-tensor.
# print("Inputs, encoded:")
# print(inputs)
# print("Inputs, decoded:")
# # Example inputs as a sentence.
# print(decode(inputs))
# # Example targets as int-tensor.
# print("Targets, encoded:")
# print(targets)
# # Example targets as a sentence.
# print("Targets, decoded:")
# print(decode(targets))

In [29]:
# There are many models available in Tensor2Tensor
# NOTE THAT THERE IS A UNIVERSAL TRANSFORMER IN HERE
registry.list_models()

['aligned',
 'attention_lm',
 'attention_lm_moe',
 'autoencoder_autoregressive',
 'autoencoder_basic',
 'autoencoder_basic_discrete',
 'autoencoder_dual_discrete',
 'autoencoder_ordered_discrete',
 'autoencoder_residual',
 'autoencoder_residual_discrete',
 'autoencoder_residual_vae',
 'autoencoder_stacked',
 'basic_fc_relu',
 'byte_net',
 'cycle_gan',
 'dense_bitwise_categorical_policy',
 'diagonal_neural_gpu',
 'distillation',
 'evolved_transformer',
 'feed_forward_categorical_policy',
 'feed_forward_cnn_small_categorical_policy',
 'feed_forward_cnn_small_categorical_policy_new',
 'gene_expression_conv',
 'glow',
 'imagetransformer',
 'imagetransformer2d',
 'imagetransformer_moe',
 'img2img_transformer',
 'img2img_transformer_block_parallel',
 'lstm_encoder',
 'lstm_seq2seq',
 'lstm_seq2seq_attention',
 'lstm_seq2seq_attention_bidirectional_encoder',
 'lstm_seq2seq_bidirectional_encoder',
 'mtf_bitransformer',
 'mtf_image_transformer',
 'mtf_res_net',
 'mtf_transformer',
 'mtf_unitran

In [30]:
# Create hparams and the model
model_name = "transformer"
hparams_set = "transformer_base"

hparams = trainer_lib.create_hparams(hparams_set, data_dir=data_dir, problem_name="translate_ende_wmt32k")

# NOTE: Only create the model once when restoring from a checkpoint; it's a
# Layer and so subsequent instantiations will have different variable scopes
# that will not match the checkpoint.
translate_model = registry.model(model_name)(hparams, Modes.EVAL)

W0713 14:20:05.167868  4136 deprecation_wrapper.py:119] From C:\Users\benar\Anaconda3\lib\site-packages\tensor2tensor\layers\common_layers.py:94: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0713 14:20:05.194844  4136 deprecation_wrapper.py:119] From C:\Users\benar\Anaconda3\lib\site-packages\tensor2tensor\utils\t2t_model.py:243: The name tf.summary.text is deprecated. Please use tf.compat.v1.summary.text instead.



In [31]:
# Copy the pretrained checkpoint locally
ckpt_name = "transformer_ende_test"
gs_ckpt = os.path.join(gs_ckpt_dir, ckpt_name)
!gsutil -q cp -R {gs_ckpt} {checkpoint_dir}
ckpt_path = tf.train.latest_checkpoint(os.path.join(checkpoint_dir, ckpt_name))
ckpt_path

'./t2t/checkpoints\\transformer_ende_test\\model.ckpt-1421000'

In [39]:
# Restore and translate!
def translate(inputs):
  encoded_inputs = encode(inputs)
  with tfe.restore_variables_on_create(ckpt_path):
    model_output = translate_model.infer(encoded_inputs)["outputs"]
  return decode(model_output)

inputs = "Hello, my name is Ben. And my partner on this project is Mark."
outputs = translate(inputs)

print("Inputs: %s" % inputs)
print("Outputs: %s" % outputs)

Inputs: Hello, my name is Ben. And my partner on this project is Mark.
Outputs: Hallo, mein Name ist Ben, und mein Partner an diesem Projekt ist Mark.


## Trying to imitate BERT

In [27]:
DATA_COLUMN = 'body'
LABEL_COLUMN = 'headline'

# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train_df.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

# test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
#                                                                    text_a = x[DATA_COLUMN], 
#                                                                    text_b = None, 
#                                                                    label = x[LABEL_COLUMN]),